In [1]:
import up42

import rasterio
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Authenticate user and initialise project.
up42.authenticate(project_id="b2ee0f3d-60ae-4d6e-859e-a1f705d3f3b4", 
                  project_api_key="5pOc3fHo.hpQyJP7C9WS7SEugbgPWHDbohUshJ3OJRHz")
project = up42.initialize_project()

2022-05-25 16:10:15,066 - Authentication with UP42 successful!
2022-05-25 16:10:15,309 - Initialized Project(name: Forestfire, project_id: b2ee0f3d-60ae-4d6e-859e-a1f705d3f3b4, description: , createdAt: 2022-05-11T19:30:23.588443Z)


In [10]:
# Define the aoi
aoi = {"type": "FeatureCollection","features": [{"type": "Feature","properties": {},
    "geometry": {"type": "Polygon","coordinates": [[[14.610298,-24.516841],
                                                    [14.622478,-24.516841],
                                                    [14.622478,-24.50833],
                                                    [14.610298,-24.50833],
                                                    [14.610298,-24.516841]]]}}]}

In [11]:
# Construct the workflow.
workflow = project.create_workflow(name="Sentinel2-l2-analytic-example", use_existing=True)
input_tasks = ["Sentinel-2 L2A Analytic (GeoTIFF)"]
workflow.add_workflow_tasks(input_tasks)

2022-05-25 16:11:45,561 - Getting existing workflows in project ...
2022-05-25 16:11:45,962 - Got 3 workflows for project b2ee0f3d-60ae-4d6e-859e-a1f705d3f3b4.
2022-05-25 16:11:46,405 - Using existing workflow: Sentinel2-l2-analytic-example - 2c7a592e-981a-4d1b-a196-9a3a103213b3
2022-05-25 16:11:48,679 - Added tasks to workflow: [{'name': 'esa-s2-l2a-gtiff-analytic:1', 'parentName': None, 'blockId': 'e13d8e92-2763-4640-80d6-1501b2729707'}]


In [12]:
# Define the aoi and input parameters of the workflow to run it.
input_parameters = workflow.construct_parameters(geometry=aoi, 
                                                 geometry_operation='intersects', 
                                                 start_date="2002-11-07", 
                                                 end_date="2020-12-31",
                                                 limit=1)
input_parameters["esa-s2-l2a-gtiff-analytic:1"].update({"max_cloud_cover":10})

In [13]:
# Price Estimation.
workflow.estimate_job(input_parameters)

2022-05-25 16:11:54,029 - Estimated: 1-1 Credits, Duration: 0-0 min.


{'esa-s2-l2a-gtiff-analytic:1': {'blockConsumption': {'resources': {'unit': 'SQUARE_KM',
    'min': 1.168825,
    'max': 1.168825},
   'credit': {'min': 0, 'max': 0}},
  'machineConsumption': {'duration': {'min': 0, 'max': 0},
   'credit': {'min': 1, 'max': 1}}}}

In [14]:
# Run test job to query data availability and check the configuration.
test_job = workflow.test_job(input_parameters, track_status=True)

2022-05-25 16:11:54,037 - +++++++++++++++++++++++++++++++++
2022-05-25 16:11:54,038 - Running this job as Test Query...
2022-05-25 16:11:54,039 - +++++++++++++++++++++++++++++++++
2022-05-25 16:11:54,039 - Selected input_parameters: {'esa-s2-l2a-gtiff-analytic:1': {'time': '2020-11-07T00:00:00Z/2020-12-31T23:59:59Z', 'limit': 1, 'max_cloud_cover': 10, 'intersects': {'type': 'Polygon', 'coordinates': (((14.610298, -24.516841), (14.622478, -24.516841), (14.622478, -24.50833), (14.610298, -24.50833), (14.610298, -24.516841)),)}}, 'config': {'mode': 'DRY_RUN'}}
2022-05-25 16:11:55,120 - Created and running new job: f7ce8545-c5cb-43bc-8e05-28d9939f52ea.
2022-05-25 16:11:55,563 - Tracking job status continuously, reporting every 30 seconds...
2022-05-25 16:12:28,024 - Job is RUNNING! - f7ce8545-c5cb-43bc-8e05-28d9939f52ea
2022-05-25 16:12:44,131 - Job finished successfully! - f7ce8545-c5cb-43bc-8e05-28d9939f52ea


In [15]:
# Run actual Job.
job = workflow.run_job(input_parameters, track_status=True)

2022-05-25 16:12:49,137 - Selected input_parameters: {'esa-s2-l2a-gtiff-analytic:1': {'time': '2020-11-07T00:00:00Z/2020-12-31T23:59:59Z', 'limit': 1, 'max_cloud_cover': 10, 'intersects': {'type': 'Polygon', 'coordinates': (((14.610298, -24.516841), (14.622478, -24.516841), (14.622478, -24.50833), (14.610298, -24.50833), (14.610298, -24.516841)),)}}}
2022-05-25 16:12:52,064 - Created and running new job: a9cbc2ad-ddca-426e-95c5-93aadf3f7725.
2022-05-25 16:12:52,310 - Tracking job status continuously, reporting every 30 seconds...
2022-05-25 16:13:24,485 - Job is RUNNING! - a9cbc2ad-ddca-426e-95c5-93aadf3f7725
2022-05-25 16:13:56,497 - Job is RUNNING! - a9cbc2ad-ddca-426e-95c5-93aadf3f7725
2022-05-25 16:14:28,296 - Job is RUNNING! - a9cbc2ad-ddca-426e-95c5-93aadf3f7725
2022-05-25 16:15:00,404 - Job is RUNNING! - a9cbc2ad-ddca-426e-95c5-93aadf3f7725
2022-05-25 16:15:34,049 - Job is RUNNING! - a9cbc2ad-ddca-426e-95c5-93aadf3f7725
2022-05-25 16:15:49,940 - Job finished successfully! - a9cb

In [16]:
# Downlod Results
job.download_results()

2022-05-25 16:16:36,452 - Downloading results of job a9cbc2ad-ddca-426e-95c5-93aadf3f7725
2022-05-25 16:16:36,454 - Download directory: C:\Users\DEEP\python_codes\Untitled Folder\project_b2ee0f3d-60ae-4d6e-859e-a1f705d3f3b4\job_a9cbc2ad-ddca-426e-95c5-93aadf3f7725
112it [00:00, 22455.16it/s]
2022-05-25 16:16:37,654 - Download successful of 14 files to output_directory 'C:\Users\DEEP\python_codes\Untitled Folder\project_b2ee0f3d-60ae-4d6e-859e-a1f705d3f3b4\job_a9cbc2ad-ddca-426e-95c5-93aadf3f7725': ['data.json', 'S2B_33JVN_20201215_0_L2A_B01.tif', 'S2B_33JVN_20201215_0_L2A_B02.tif', 'S2B_33JVN_20201215_0_L2A_B03.tif', 'S2B_33JVN_20201215_0_L2A_B04.tif', 'S2B_33JVN_20201215_0_L2A_B05.tif', 'S2B_33JVN_20201215_0_L2A_B06.tif', 'S2B_33JVN_20201215_0_L2A_B07.tif', 'S2B_33JVN_20201215_0_L2A_B08.tif', 'S2B_33JVN_20201215_0_L2A_B09.tif', 'S2B_33JVN_20201215_0_L2A_B11.tif', 'S2B_33JVN_20201215_0_L2A_B12.tif', 'S2B_33JVN_20201215_0_L2A_B8A.tif', 'S2B_33JVN_20201215_0_L2A_vrt.vrt']


['C:\\Users\\DEEP\\python_codes\\Untitled Folder\\project_b2ee0f3d-60ae-4d6e-859e-a1f705d3f3b4\\job_a9cbc2ad-ddca-426e-95c5-93aadf3f7725\\data.json',
 'C:\\Users\\DEEP\\python_codes\\Untitled Folder\\project_b2ee0f3d-60ae-4d6e-859e-a1f705d3f3b4\\job_a9cbc2ad-ddca-426e-95c5-93aadf3f7725\\S2B_33JVN_20201215_0_L2A\\S2B_33JVN_20201215_0_L2A_B01.tif',
 'C:\\Users\\DEEP\\python_codes\\Untitled Folder\\project_b2ee0f3d-60ae-4d6e-859e-a1f705d3f3b4\\job_a9cbc2ad-ddca-426e-95c5-93aadf3f7725\\S2B_33JVN_20201215_0_L2A\\S2B_33JVN_20201215_0_L2A_B02.tif',
 'C:\\Users\\DEEP\\python_codes\\Untitled Folder\\project_b2ee0f3d-60ae-4d6e-859e-a1f705d3f3b4\\job_a9cbc2ad-ddca-426e-95c5-93aadf3f7725\\S2B_33JVN_20201215_0_L2A\\S2B_33JVN_20201215_0_L2A_B03.tif',
 'C:\\Users\\DEEP\\python_codes\\Untitled Folder\\project_b2ee0f3d-60ae-4d6e-859e-a1f705d3f3b4\\job_a9cbc2ad-ddca-426e-95c5-93aadf3f7725\\S2B_33JVN_20201215_0_L2A\\S2B_33JVN_20201215_0_L2A_B04.tif',
 'C:\\Users\\DEEP\\python_codes\\Untitled Folder\\proj

In [ ]:
# Identify the RGB bands.
red = 'project_1f2bad99-daad-4a60-9192-5169263caff6/job_01bdf71e-7c35-4454-844b-809900bcef64/S2B_33JVN_20201215_0_L2A_B04.tif'
green = 'project_1f2bad99-daad-4a60-9192-5169263caff6/job_01bdf71e-7c35-4454-844b-809900bcef64/S2B_33JVN_20201215_0_L2A_B03.tif'
blue = 'project_1f2bad99-daad-4a60-9192-5169263caff6/job_01bdf71e-7c35-4454-844b-809900bcef64/S2B_33JVN_20201215_0_L2A_B02.tif'
file_list = [red, green, blue]

In [ ]:
def normalize(array):
    """Normalizes numpy arrays into scale 0.0 - 1.0"""
    array_min, array_max = array.min(), array.max()
    return ((array - array_min)/(array_max - array_min))